<a href="https://colab.research.google.com/github/arstotrix/kursovaya2020_final/blob/master/clustering_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deeppavlov

     |████████████████████████████████| 952kB 2.8MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 6.7MB 43.1MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 1.5MB 45.7MB/s 
     |████████████████████████████████| 665kB 42.5MB/s 
     |████████████████████████████████| 8.0MB 41.7MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 7.3MB 15.6MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 512kB 41.8MB/s 
     |████████████████████████████████| 10.4MB 40.9MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 860kB 37.6MB/s 
     |████████████████████████████████| 2.1MB 38.1MB/s 
     |

In [ ]:
!pip install hdbscan

     |████████████████████████████████| 4.7MB 2.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2354327 sha256=9205bad8cd5f946ec94d894d988715526b9dbd7a0176a1d017debbfd2536b900
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan


In [ ]:
!pip install tqdm

In [ ]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 34kB/s 
     |████████████████████████████████| 3.8MB 2.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 512kB 27.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=916030d95f759f9b6822ffc2086db989a2ccb9bab91dd1f5c500727a2676bbe5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing in

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

#Graphs
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Pre-trained BERT and ELMo for Russian
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

#Grid search to find optimal parameters
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import GridSearchCV

# Dimension reduction and clustering libraries
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


In [ ]:
#tqdm.pandas()

# progress bar
from tqdm import tqdm, tqdm_notebook

# instantiate
tqdm.pandas(tqdm_notebook)

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [ ]:
def remove_target_word(context, positions):
    positions = [i.split('-') for i in positions.split(',')]
    for position in positions:
        start = int(position[0])
        end = int(position[1])
        return context.replace(context[start:end], '')

In [ ]:
def elmo_preprocess(text):
    result = []
    for sentence in text.split('.'):
        tokens = [re.sub('[^А-Яа-я.!? ]', ' ', token) for token in word_tokenize(sentence) if token not in russian_stopwords\
              and token != " " and token != "" \
              and token.strip() not in punctuation]
        result.append(tokens)
    return [x for x in result if x]

In [ ]:
def bert_preprocess(text):
    result = []
    for sentence in text.split('.'):
        tokens = [re.sub('[^А-Яа-я.!? ]', ' ', token) for token in word_tokenize(sentence) if token not in russian_stopwords\
              and token != " " and token != "" \
              and token.strip() not in punctuation]
        sentence = ' '.join(tokens)
        result.append(sentence)
    return [x for x in result if x]

In [ ]:
#Download ELMO model for Russian
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz",\
                    elmo_output_names=["elmo"])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#Download RUBERT configs
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['download'][0]['url'] = 'http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz'
bert_config['metadata']['variables']['BERT_PATH'] = '{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_pt'

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.7MB/s 
     |████████████████████████████████| 1.1MB 18.1MB/s 
     |████████████████████████████████| 3.0MB 18.4MB/s 


In [ ]:
import transformers

In [ ]:
#загружаем модель BERT
bert = build_model(bert_config, download=True)

2020-10-10 22:25:46.329 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_pt.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_pt.tar.gz
100%|██████████| 662M/662M [02:46<00:00, 3.98MB/s]
2020-10-10 22:28:33.438 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_pt.tar.gz archive into /root/.deeppavlov/downloads/bert_models
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_pt.tar.gz archive into /root/.deeppavlov/downloads/bert_models


In [ ]:
'''bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'path/to/bert/directory'

m = build_model(bert_config)

texts = ['Hi, i want my embedding.', 'And mine too, please!']
tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)'''

"bert_config = read_json(configs.embedder.bert_embedder)\nbert_config['metadata']['variables']['BERT_PATH'] = 'path/to/bert/directory'\n\nm = build_model(bert_config)\n\ntexts = ['Hi, i want my embedding.', 'And mine too, please!']\ntokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)"

In [ ]:
help(bert)

Help on Chainer in module deeppavlov.core.common.chainer object:

class Chainer(deeppavlov.core.models.component.Component)
 |  Builds a component pipeline from heterogeneous components (Rule-based/ML/DL). It allows to train
 |  and infer models in a pipeline as a whole.
 |  
 |  Attributes:
 |      pipe: list of components and their input and output variable names for inference
 |      train_pipe: list of components and their input and output variable names for training and evaluation
 |      in_x: names of inputs for pipeline inference mode
 |      out_params: names of pipeline inference outputs
 |      in_y: names of additional inputs for pipeline training and evaluation modes
 |      forward_map: list of all variables in chainer's memory after  running every component in ``self.pipe``
 |      train_map: list of all variables in chainer's memory after  running every component in ``train_pipe.pipe``
 |      main: reference to the main component
 |  
 |  Args:
 |      in_x: names of i

In [ ]:
def extract_token_vectors(bert_context_vectors, context, positions):
    """ Function to extract BERT word embeddings from BERT-embedder's output"""
    words = []
    vectors = []
    positions = [i.split('-') for i in positions.split(',')]
    for position in positions:
        start = int(position[0])
        end = int(position[1])
        words.append(context[start:end+1])
    for word in words:
        for list_idx, token_list in enumerate(bert_context_vectors[0]):
            if word in token_list:
                word_idx = token_list.index(word)
                bert_token_vector = bert_context_vectors[1][list_idx][word_idx]
                vectors.append(bert_token_vector)
    try:
        result = np.vstack(vectors)
    except ValueError:
        result = None
    return result

In [ ]:
def elmo_vectorize(elmo_preprocessed_text):
    return elmo(elmo_preprocessed_text)

def bert_vectorize(bert_preprocessed_text):
    return(bert(bert_preprocessed_text))

In [ ]:
def average_pooler_outputs_mean(vectors):
    """ Function to average BERT pooler outputs """
    average_vector = np.array([sum(subvector) / len(subvector) for subvector in vectors[6].transpose()])
    return average_vector

In [ ]:
def stack_vectors(bert_embeddings):
    vectors = []
    for i, vector in enumerate(bert_embeddings):
        vectors.append(vector[0])
        
    X = np.vstack(vectors)
    return X

In [ ]:
#Dim reduction experiments
reducer = umap.UMAP()
standard_embedding = reducer.fit_transform(X)
standard_embedding.shape

NameError: ignored

In [ ]:
#Let's see what we have
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1],\
            c=[sns.color_palette()[x] for x in X_gold_senses], cmap='Spectral')

# plt.title("UMAP projection of BERT embeddings of word 'белок'", fontsize=24);

In [ ]:
def find_parameters(X):
    """ Function to find optimal parameters for Affinity propagation algorithm"""
    S =-pairwise_distances(X,metric=affinity, squared=True)
    param_grid   =  np.unique(map(int, np.linspace(np.min(S), np.median(S), 30)))
    search = GridSearchCV(AffinityPropagation(), param_grid, verbose=0) 
    return search.fit(X)

In [ ]:
def cluster_via_affinity_prop(matrix, gold_senses=list_of_dom_gold_senses, preference=preference):
    """ Clustering via Affinity propagation algorithm """
    af = AffinityPropagation(preference=preference).fit(matrix)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_
    n_clusters_ = len(cluster_centers_indices)

    print('Estimated number of clusters: %d' % n_clusters_)
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(gold_senses, labels))
    print("Completeness: %0.3f" % metrics.completeness_score(gold_senses, labels))
    print("V-measure: %0.3f" % metrics.v_measure_score(gold_senses, labels))
    print("Adjusted Rand Index: %0.3f"
          % metrics.adjusted_rand_score(gold_senses, labels))
    print("Adjusted Mutual Information: %0.3f"
          % metrics.adjusted_mutual_info_score(gold_senses, labels))
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(matrix, labels, metric='sqeuclidean'))

In [ ]:
def cluster_word_senses(df, preference):
    """ Final function to cluster word meanings """
    predicted = []
    gold_senses = []
    for query in df.word.unique():
        print('Now analyzing', query, '...')
        df_word = df[df['word'] == query]
        vectors = []
        for i, vector in enumerate(df_word.target_words_bert_embs):
            vectors.append(vector[0])
        X = np.vstack(vectors)
        print('emb shape: ',X.shape)
        X_gold_senses = [int(i) for i in df_word.gold_sense_id]
        min_gold = min(X_gold_senses)
        X_gold_senses = [i - min_gold for i in X_gold_senses]
        print('gold', X_gold_senses)
        print('len gold_senses: ', len(X_gold_senses))
        gold_senses += X_gold_senses
        gold_senses = [str(i) for i in gold_senses]
        
        clustering = AffinityPropagation(preference=preference).fit(X)
        cur_predicted = clustering.labels_.tolist()
        predicted += cur_predicted
        predicted = [str(i) for i in predicted]
    
    df.predict_sense_id = predicted
    df.gold_senses = gold_senses
    return df

In [ ]:
def ari_per_word_weighted(df):
    """ Function for final evaluation. """
    
    words = {word: (adjusted_rand_score(df_word.gold_sense_id, df_word.predict_sense_id), len(df_word))
             for word in df.word.unique()
             for df_word in (df.loc[df['word'] == word],)}
    
    print(words)

    cumsum = sum(ari * count for ari, count in words.values())
    total  = sum(count for _, count in words.values())
    
    print(cumsum, total)

    assert total == len(df) 

    return cumsum / total, words

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = os.getcwd() + '/drive/My Drive/untagged/'
files = os.listdir(path)
file = path + files[0]
with open (file, encoding='utf-8') as f:
  sentences = f.read().splitlines()
  tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = bert(sentences)

print(tokens[0])
print(token_embs[0])

['Её', 'завернули', 'в', 'кусок', 'сероватой', 'бумаги', 'и', 'упаковали', 'в', 'прозрачный', 'мешочек', '.']
[[ 0.10478529 -0.7811744   1.5699083  ...  0.53593516  0.17087647
   0.01380411]
 [ 0.46247905 -0.8807056   0.50503373 ...  0.32085255 -0.5700146
  -0.6682901 ]
 [ 1.605512   -1.0558653  -0.38912356 ...  1.8150644  -0.12538502
  -1.094595  ]
 ...
 [ 0.33373445 -0.10884144  0.15312503 ...  0.2741692   0.09904095
  -0.38784984]
 [-0.44039008  0.24926904 -0.8269304  ...  0.08725259 -0.20478015
   0.517728  ]
 [ 0.2973631  -0.42974395  0.71850234 ...  0.38392076  0.09275401
  -0.2803605 ]]


In [ ]:
file = path + files[1]
with open (file, encoding='utf-8') as f:
  sentences = f.read().splitlines()
  tokens_1, token_embs_1, subtokens_1, subtoken_embs_1, sent_max_embs_1, sent_mean_embs_1, bert_pooler_outputs_1 = bert(sentences)
print(tokens_1[0])
print(token_embs_1[0])

['\ufeffС', 'ностальгией', 'вспоминаю', 'только', 'универ', ',', 'это', 'особый', 'мир', ',', 'по', 'которому', 'потом', 'скучаешь', '.']
[[-0.14588007 -0.185153    0.6619032  ...  0.28623748  0.38101152
   0.07768197]
 [-0.02604854 -0.07009289 -0.02454762 ...  0.34504956  0.32805717
  -0.2177502 ]
 [ 0.3318938  -0.00854438  0.09116171 ... -0.7021017  -0.23567672
   0.28729188]
 ...
 [ 0.29097483 -0.5836606   0.25092518 ... -0.57325196 -0.05737009
  -0.0649939 ]
 [ 0.50509393 -0.17283788 -0.38833037 ... -0.92600673 -0.9555784
   0.19540757]
 [ 0.27181438 -0.37938774  0.3942691  ...  0.02811895  0.11207419
  -0.18031313]]


In [ ]:
file = path + files[2]
with open (file, encoding='utf-8') as f:
  sentences = f.read().splitlines()
  tokens_2, token_embs_2, subtokens_2, subtoken_embs_2, sent_max_embs_2, sent_mean_embs_2, bert_pooler_outputs_2 = bert(sentences)
print(tokens_2[0])
print(token_embs_2[0])

['Кто', 'же', 'знал', ',', 'что', 'когда', '-', 'нибудь', 'их', 'столкнёт', 'судьба', 'в', 'лице', 'нового', 'сотрудника', '—', 'Самохвалова', '…', '«', '…', 'Пламя', 'скорби', 'свистит', 'под', 'ногами', ',', '/', 'Ворохами', 'листвы', 'шевеля', '…', '»', 'В', '«', 'Статистическое', 'учреждение', '»', 'назначили', 'нового', '«', 'главного', '»', ',', 'которым', 'оказывается', 'старый', 'друг', 'одной', 'из', 'сотрудниц', '—', 'Рыжовой', '.']
[[-0.92314786 -0.01130431  0.68326384 ...  0.38482624 -0.27290708
  -0.07632419]
 [ 0.3012279  -0.564792   -0.76994497 ...  0.7971826  -0.29860768
  -0.30226147]
 [ 0.5594669  -0.40273002  0.08004247 ...  0.40653935 -0.09368179
  -0.42906556]
 ...
 [-0.41394758 -0.28846028  0.1755642  ...  1.5149912   0.65843076
   0.08524976]
 [-0.5662451  -0.4801915  -0.3071556  ...  0.50742614  0.4585031
  -0.02253559]
 [ 0.1304772  -0.8480271  -0.12245673 ...  0.45152247 -0.20638594
   0.12305369]]


In [ ]:
file = path + files[3]
with open (file, encoding='utf-8') as f:
  sentences = f.read().splitlines()
  tokens_3, token_embs_3, subtokens_3, subtoken_embs_3, sent_max_embs_3, sent_mean_embs_3, bert_pooler_outputs_3 = bert(sentences)
print(tokens_3[0])
print(token_embs_3[0])

['На', 'данном', 'временном', 'интервале', 'наши', 'пристрастия', '—', 'горные', ',', 'двухподвесочные', ',', 'чтобы', 'и', 'по', 'лестницам', 'супермаркета', 'можно', 'было', 'спуститься', ',', 'и', 'сквозь', 'лес', 'пробиться', ',', 'и', 'на', 'трассе', 'двадцать', 'километров', 'одолеть', 'без', 'усталости', '.']
[[ 0.4358259  -0.6092326   1.0631086  ...  1.4777629   0.34908387
   0.01570448]
 [ 0.6002555  -1.0388457   0.6063347  ...  0.9035062  -0.02797903
   0.31063744]
 [ 0.17498188  0.5650411   0.9754694  ...  1.406402   -0.31218663
  -1.1108836 ]
 ...
 [ 0.40839773  0.3407201  -0.5083811  ... -0.15181172 -0.5281812
   0.2832448 ]
 [ 0.34586433  0.1529005   0.5693935  ... -0.3467772  -0.8774383
  -0.5758203 ]
 [ 0.17596379 -0.19243076  0.29428944 ...  0.0981304  -0.01304511
  -0.37157515]]


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def clusterfrick(tokens, token_embs, word, cluster_number):
  clusterlist = []
  sentlist = []
  for sent_index, sentence in enumerate(tokens):
    for token_index, token in enumerate(sentence):
      if word == token:
        clusterlist.append(token_embs[sent_index][token_index])
        sentlist.append(sentence)


  clusterer = KMeans(n_clusters = cluster_number)
  clusterer = clusterer.fit(clusterlist)
  clusters = clusterer.predict(clusterlist)
  print(len(clusters))
  return clusters,sentlist

In [ ]:
transp,transp_s = clusterfrick(tokens, token_embs, 'прозрачный', 4)
world, world_s = clusterfrick(tokens_1, token_embs_1, 'мир', 7)
fate, fate_s = clusterfrick(tokens_2, token_embs_2, 'судьба', 4)
forest, forest_s = clusterfrick(tokens_3, token_embs_3, 'лес', 4)

279
487
367
552


In [ ]:
def tablemaker(xx, xx_s):
  for stnc_index, stnc in enumerate(xx_s): 
    stnc = ' '.join(stnc)
    line = stnc + '\t' + str(xx[stnc_index]) 
    print(line)

In [ ]:
tablemaker(world, world_s)

﻿С ностальгией вспоминаю только универ , это особый мир , по которому потом скучаешь .	3
Но , опять же , мир изменился …	5
Была в США , в Washington DC и штате Вермонт , как делегат программы « Открытый мир » по теме « Образование » .	6
Из той же оперы , когда российский профессор в интернете на весь мир заявляет , что он работает в Германии в университете , при детальном же рассмотрении оказывается , что он работает там на так называемом 1 , - Евро джоб ( т . е . за 1 , - Евро / Час ) .	4
В дыре , в заводской общаге , и мир их был ограничен этим .	5
Кстати , стремящимся и мир стал помогать ( японец закодировал пьяницу - отца от алкоголя , помог им культурно развиться , пригласив в Японию , гаишник обеспечил сопровождение дедуле в его экстрим - поездке , когда тот прицепился альпинистским снаряжением к самосвалу ) .	5
Пойми , Петя , мир состоит из умных и глупых , из сильных и слабых , из тех , кто может свою судьбу перевернуть и из тех , кто плывет по течению .	5
Друзьям не отказывают